## Project Title: To determine if data augmentation using the method proposed in 'Finding Order in Chaos: A Novel Data Augmentation Method for Time Series in Contrastive Learning' will lead to better 1 day prediction results.



In [1]:
import numpy as np
import tensorflow as tf
import random
import os

# Seed value
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)


In [2]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
import seaborn as sns
from tensorflow.keras import layers, Model
import numpy as np
import torch
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.fft import rfft, rfftfreq, irfft
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
# from tcn import TCN  # If you have the tcn p /ackage installed
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.seasonal import STL
from sklearn.utils import resample

import optuna
from optuna.samplers import TPESampler
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from pykalman import KalmanFilter
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

### Initialize Parameters

In [3]:
# Assuming cfg is a configuration object with a seed attribute
cfg = type('config', (object,), {'seed': 42})
# Make the sampler behave in a deterministic way.
sampler = TPESampler(seed=cfg.seed)

alpha = 0.4
seq_len = 20
test_size = 0.3

### Helper Functions

In [4]:
# Function to import stock data
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

def z_score_normalize(series):
    mean = series.mean()
    std = series.std()
    return (series - mean) / std

def denormalize_z_score(normalized_series, original_mean, original_std):
    return (normalized_series * original_std) + original_mean

# Function to create model (make sure this is defined in your environment)
def create_model(best_params, input_shape):
    model = Sequential()
    model.add(LSTM(best_params['lstm_units'], input_shape=input_shape, return_sequences=True))
    model.add(Dropout(best_params['dropout_rate']))
    model.add(LSTM(best_params['lstm_units']))  # Stacking LSTM for deep learning
    model.add(Dropout(best_params['dropout_rate']))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mse')
    return model

def create_sequences(features, target, seq_len):
    X, y = [], []
    for i in range(len(target) - seq_len):
        X.append(features[i:(i + seq_len)])
        y.append(target[i + seq_len])
    return np.array(X), np.array(y)

In [5]:
def plot_correlation(df):
    correlation_matrix = df.corr()

    # Set up the matplotlib figure
    plt.figure(figsize=(10, 8))

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm',
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

    # Adjust the plot as needed
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()  # Adjusts the plot to ensure everything fits without overlap

    # Show the plot
    plt.show()

In [6]:
def engineer_features(data):
    df = data.copy(deep=True)
    delta = df['Close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.rolling(window=14).mean()
    roll_down = down.abs().rolling(window=14).mean()
    RS = roll_up / roll_down
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))

    # Volume Weighted Average Price (VWAP)
    vwap = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
    df['VWAP'] = vwap

    # Price Ratios
    df['high_to_low_ratio'] = df['High'] / df['Low']
    df['open_to_close_ratio'] = df['Open'] / df['Close']

    # Volatility
    df['volatility_10'] = df['Close'].rolling(window=10).std()

    df1 = df.drop(columns=['Open', 'High', 'Low', 'Adj Close']).dropna()
    return df1

In [7]:
# def cut_mix(data1, data2, alpha=0.2):
#     assert len(data1) == len(data2)
#     size = len(data1)
#     cut_point = np.random.randint(0, size)
#     cut_length = int(size * alpha)
    
#     mixed_data = np.copy(data1)
#     mixed_data[cut_point:cut_point+cut_length] = data2[cut_point:cut_point+cut_length]
    
#     return mixed_data

def cut_mix(df1, df2, alpha=0.2):
    np.random.seed(42)
    
    assert df1.shape == df2.shape
    size = len(df1)
    cut_point = np.random.randint(0, size,)
    cut_length = int(size * alpha)
    
    mixed_df = df1.copy()
    mixed_df.iloc[cut_point:cut_point+cut_length] = df2.iloc[cut_point:cut_point+cut_length]
    
    return mixed_df

def binary_mix(data1, data2, alpha=0.2):
    np.random.seed(42)
    
    assert len(data1) == len(data2)
    size = data1.shape
    mask = np.random.binomial(1, alpha, size=size).astype(bool)
    
    mixed_data = np.where(mask, data1, data2)
    
    return pd.DataFrame(mixed_data, columns=data1.columns)

def linear_mix(data1, data2, alpha=0.2):
    assert len(data1) == len(data2)
    
    mixed_data = alpha * data1 + (1 - alpha) * data2
    
    return mixed_data

def geometric_mix(data1, data2, alpha=0.2):
    assert len(data1) == len(data2)
    
    mixed_data = data1**alpha * data2**(1 - alpha)
    
    return mixed_data

def amplitude_mix(data1, data2, alpha=0.2):
    assert len(data1) == len(data2)
    
    # Apply Fourier Transform to each column
    fft1 = np.fft.rfft(data1, axis=0)
    fft2 = np.fft.rfft(data2, axis=0)
    
    # Mix the magnitudes
    magnitude1 = np.abs(fft1)
    magnitude2 = np.abs(fft2)
    mixed_magnitude = alpha * magnitude1 + (1 - alpha) * magnitude2
    
    # Keep the phase of the first data
    phase1 = np.angle(fft1)
    mixed_fft = mixed_magnitude * np.exp(1j * phase1)
    
    # Perform the inverse FFT and ensure the result is two-dimensional
    mixed_data = np.fft.irfft(mixed_fft, axis=0)
    if mixed_data.ndim == 1:
        mixed_data = mixed_data.reshape(-1, 1)  # Reshape if the data is one-dimensional
    
    # Return a DataFrame with the same column names as data1
    return pd.DataFrame(mixed_data, columns=data1.columns)


### PROPOSE TECHNIQUE BELOW

def proposed_mixup(df1, df2, threshold=0.1, alpha=0.5):
    
    def proposed_mixup_feature(data1, data2, threshold, alpha):
        
        def get_significant_frequencies(data, threshold):
            """
            Perform Fourier Transform on data and identify frequencies with significant amplitude.

            Args:
            - data: Time series data.
            - threshold: Threshold for significance, relative to the max amplitude.

            Returns:
            - significant_freq: Frequencies with significant amplitude.
            - significant_ampl: Amplitude of the significant frequencies.
            - full_spectrum: Full Fourier spectrum for all frequencies.
            """
            # Perform Fourier Transform
            spectrum = rfft(data)
            frequencies = rfftfreq(data.size, d=1)  # Assuming unit time interval between samples

            # Find significant amplitudes
            amplitude = np.abs(spectrum)
            significant_indices = amplitude > (amplitude.max() * threshold)
            significant_freq = frequencies[significant_indices]
            significant_ampl = amplitude[significant_indices]

            return significant_freq, significant_ampl, spectrum

        def phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2, alpha):
            mixed_spectrum = np.copy(spectrum1)
            freqs1 = rfftfreq(spectrum1.size, d=1)
            freqs2 = rfftfreq(spectrum2.size, d=1)

            for freq in sig_freq1:
                index1 = np.argmin(np.abs(freqs1 - freq))
                index2 = np.argmin(np.abs(freqs2 - freq))

                if index1 >= len(sig_ampl1) or index2 >= len(sig_ampl2):
                    continue  # Skip the frequency if the index is out of bounds

                phase1 = np.angle(spectrum1[index1])
                phase2 = np.angle(spectrum2[index2])

                phase_diff = (phase2 - phase1) % (2 * np.pi)
                phase_diff = phase_diff - 2 * np.pi if phase_diff > np.pi else phase_diff

                new_amplitude = alpha * sig_ampl1[index1] + (1 - alpha) * sig_ampl2[index2]
                new_phase = phase1 + alpha * phase_diff

                mixed_spectrum[index1] = new_amplitude * np.exp(1j * new_phase)

            return mixed_spectrum


        def reconstruct_time_series(mixed_spectrum):
            """
            Reconstruct time series from mixed spectrum using inverse Fourier Transform.

            Returns:
            - mixed_time_series: The reconstructed time series.
            """
            # Perform inverse Fourier Transform
            mixed_time_series = irfft(mixed_spectrum)

            return mixed_time_series

        # Step 1: Get significant frequencies and amplitude for both time series
        sig_freq1, sig_ampl1, spectrum1 = get_significant_frequencies(data1, threshold)
        sig_freq2, sig_ampl2, spectrum2 = get_significant_frequencies(data2, threshold)

        # Step 2: Identify significant frequencies (already done in step 1)

        # Step 3: Phase and Magnitude Mixup
        mixed_spectrum = phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2, alpha)

        # Step 4: Reconstruction of the time series
        mixed_time_series = reconstruct_time_series(mixed_spectrum)
        return mixed_time_series
    
    output_df = pd.DataFrame()
    
    for feature in df1.columns:
        output_df[feature] = proposed_mixup_feature(df1[feature].values, df2[feature].values, threshold, alpha)
        
    return output_df

In [8]:
def objective(trial):
    # Hyperparameters to be tuned by Optuna
    n_layers = trial.suggest_int('n_layers', 1, 3)
    lstm_units = trial.suggest_categorical('lstm_units', [50, 100, 150])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    epochs = trial.suggest_int('epochs', 20, 100)
    
    # Dictionary to hold RMSE for each stock
    stock_rmse = {}
    
    for stock, df in historical_data_augmented.items():
        # Preprocess the data
        df = df.copy()
        rets = df['Close'].pct_change().dropna()
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(df[df.columns].values)
        seq_len = 20
        X, y = create_sequences(scaled_features, rets.values, seq_len)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        # Model architecture
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = Sequential()
        for i in range(n_layers):
            model.add(LSTM(units=lstm_units, return_sequences=(i < n_layers - 1)))
            model.add(Dropout(rate=dropout_rate))
        model.add(Dense(units=1))
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
        
        # Fit the model
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        
        # Predictions and evaluate
        predictions = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        stock_rmse[stock] = rmse
    
    # Calculate the average RMSE across all stocks
    average_rmse = np.mean(list(stock_rmse.values()))
    
    return average_rmse

In [9]:
def create_augmented_data(df1, df2, method, alpha=0.2):
    if method == 'cut_mix':
        df = cut_mix(df1, df2, alpha)
    elif method == 'binary_mix':
        df = binary_mix(df1, df2, alpha)
    elif method == 'linear_mix':
        df = linear_mix(df1, df2, alpha)
    elif method == 'geometrix_mix':
        df = geometric_mix(df1, df2, alpha)
    elif method == 'amplitude_mix':
        df = amplitude_mix(df1, df2, alpha)
    elif method == 'proposed_mix':
        df = proposed_mixup(df1, df2, alpha)

    # Original
    else:
        df = df1.copy()
        
    return df

In [10]:
def plot_TSNE(df1, df2):
    df1.columns = df1.columns.astype(str)
    df2.columns = df2.columns.astype(str)

    df1_log = np.log(df1 + 1)  # Adding 1 to avoid log(0)
    df2_log = np.log(df2 + 1)

    combined_data = pd.concat([df1_log, df2_log])

    # Apply t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=100, n_iter=1000, init='pca')
    tsne_results = tsne.fit_transform(combined_data)

    # Now we split the t-SNE results back into original and augmented parts
    tsne_df1 = tsne_results[:len(df1), :]
    tsne_df2 = tsne_results[len(df1):, :]

    # Plot the results
    plt.figure(figsize=(12,8))
    plt.scatter(tsne_df1[:, 0], tsne_df1[:, 1], label='Original', alpha=0.5)
    plt.scatter(tsne_df2[:, 0], tsne_df2[:, 1], label='Augmented', alpha=0.5)
    plt.legend()
    plt.show()

In [11]:
from sklearn.feature_selection import mutual_info_regression
def calculate_entropy(variable):
    value,counts = np.unique(variable, return_counts=True)
    return entropy(counts, base=2)

# Function to calculate normalized mutual information
def calculate_normalized_mi(variable_1, variable_2):
    mi = mutual_info_score(variable_1, variable_2)
    entropy_1 = calculate_entropy(variable_1)
    entropy_2 = calculate_entropy(variable_2)
    # Normalizing by the average entropy
    normalized_mi = mi / ((entropy_1 + entropy_2) / 2)
    return normalized_mi

def calculate_MI(original, augmented):
# Assuming df_original and df_augmented are your dataframes
    for column in original.columns:
        # Ensure the data is in the correct format, e.g., continuous or discrete
        # For continuous variables, you'd typically bin them before calculating mutual information
        original_data = original[column].to_numpy()
        augmented_data = augmented[column].to_numpy()

        # Calculate normalized MI for each column
        normalized_mi = calculate_normalized_mi(original_data, augmented_data)
        print(f'Normalized Mutual Information for {column}: {normalized_mi}')


In [12]:
from statsmodels.nonparametric.smoothers_lowess import lowess
def apply_lowess_smoothing(df, frac=0.1):
    smoothed_data = pd.DataFrame(index=df.index)
    
    # Apply LOWESS to each column
    for column in df.columns:
        smoothed_values = lowess(df[column], df.index, frac=frac, return_sorted=False)
        smoothed_data[column] = smoothed_values
    
    return smoothed_data

In [13]:
def compare_distributions(original_df, augmented_df, features):
    num_features = len(features)
    fig, axes = plt.subplots(nrows=num_features, ncols=2, figsize=(15, 5 * num_features))

    for i, feature in enumerate(features):
        # Original Data
        sns.histplot(original_df[feature], kde=True, color="blue", ax=axes[i, 0])
        axes[i, 0].set_title(f'Original {feature}')
        
        # Augmented Data
        sns.histplot(augmented_df[feature], kde=True, color="orange", ax=axes[i, 1])
        axes[i, 1].set_title(f'Augmented {feature}')
        
    plt.tight_layout()
    plt.show()

### Pull Data from Yahoo Finance

In [14]:
start_date = '2010-01-01'
end_date = '2023-01-01'

# Define the list of Dow Jones Industrial Average companies
tickers = [
    "MMM", "AXP", "AMGN", "AAPL", "BA", "CAT", "CVX", "CSCO", "KO", "DIS"
    , "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "MCD", "MRK",
    "MSFT", "NKE", "PG", "CRM", "TRV", "UNH", "V", "WBA", "WMT"
]

# tickers = ['AAPL']
# Create a dictionary to store historical data for each company
historical_data = {}

# Loop through the Dow companies and retrieve historical data
for ticker in tickers:
    stock_data = get_stock_data(ticker, start_date, end_date)
    historical_data[ticker] = stock_data

In [15]:
def jittering(ts, noise_level=0.05):
    np.random.seed(42)
    noise = np.random.normal(loc=0, scale=noise_level, size=len(ts))
    return pd.Series(ts + noise)

def flipping(ts):
    return pd.Series(np.flip(ts))

def scaling(ts, scaling_factor=1.5):
    return pd.Series(ts * scaling_factor)

def magnitude_warping(ts, sigma=0.2, knot=4):
    np.random.seed(42)
    from scipy.interpolate import CubicSpline
    random_warps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, ))
    indices = np.linspace(0, len(ts)-1, num=knot+2)
    sp = CubicSpline(indices, random_warps)
    warp_values = sp(np.arange(len(ts)))
    return pd.Series(ts * warp_values)

def permutation(ts, n_segments=5):
    np.random.seed(42)
    permutated_ts = np.copy(ts)
    segments = np.array_split(permutated_ts, n_segments)
    np.random.shuffle(segments)
    return pd.Series(np.concatenate(segments))

def time_warping(ts, sigma=0.2, knot=4):
    np.random.seed(42)
    from scipy.interpolate import CubicSpline
    time_steps = np.arange(ts.shape[0])
    random_steps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, ts.shape[1]))
    indices = np.linspace(0, len(ts)-1, num=knot+2)
    sp = CubicSpline(indices, random_steps)
    warp_values = sp(time_steps)
    return pd.Series(warp_values * ts)

def stl_augment(data, period=61):
    ts = data.asfreq('B')
    ts = ts.interpolate()
    # Apply STL decomposition
    stl = STL(ts, seasonal=period) 
    result = stl.fit()
    seasonal, trend, remainder = result.seasonal, result.trend, result.resid
    bootstrapped_remainder = resample(remainder, replace=True, n_samples=len(remainder), random_state=42)
    bootstrapped_remainder.index = ts.index
    augmented_signal = trend + seasonal + bootstrapped_remainder
    augmented_signal = np.maximum(augmented_signal, 0)
    augmented_signal = augmented_signal[data.index]
    return augmented_signal

# Function to plot original and augmented series
def plot_augmented_ts(original_ts, augmented_ts, title='Time Series Augmentation'):
    augmented_ts.index = original_ts.index
    plt.figure(figsize=(14, 6))
    plt.plot(original_ts, label='Original')
    plt.plot(augmented_ts, label='Augmented')
    plt.title(title)
    plt.legend()
    plt.show()

### AAPL

In [16]:
aapl = historical_data['AAPL']
aapl = engineer_features(aapl)
aapl

In [17]:
def augment_dataframe(stock_df, method):
    augmented_stockdf = pd.DataFrame()
    for col in stock_df.columns:
        val = aapl[col]
        if method == 'jittering':
            aug_val = jittering(val)
        elif method == 'flipping':
            aug_val = flipping(val)
        elif method == 'scaling':
            aug_val = scaling(val)
        elif method == 'magnitude_warping':
            aug_val = magnitude_warping(val)
        elif method == 'permutation':
            aug_val = permutation(val)
        elif method == 'stl_augment':
            aug_val = stl_augment(val)

        augmented_stockdf[col] = aug_val
    return augmented_stockdf

In [18]:
lowess = apply_lowess_smoothing(aapl)
cut_lowess = create_augmented_data(aapl, lowess, method='cut_mix')
binary_lowess = create_augmented_data(aapl, lowess, method='binary_mix')
linear_lowess = create_augmented_data(aapl, lowess, method='linear_mix')
geometric_lowess = create_augmented_data(aapl, lowess, method='geometrix_mix')
amplitude_lowess = create_augmented_data(aapl, lowess, method='amplitude_mix')
proposedmix_lowess = create_augmented_data(aapl, lowess, method='proposed_mix')

jittered_ts = augment_dataframe(aapl, 'jittering')
flipped_ts = augment_dataframe(aapl, 'flipping')
scaled_ts = augment_dataframe(aapl, 'scaling')
mag_warped_ts = augment_dataframe(aapl, 'magnitude_warping')
permuted_ts = augment_dataframe(aapl, 'permutation')
stl_ts = augment_dataframe(aapl, 'stl_augment')

augmented_datasets = {
#     'cut_mix': cut_lowess,
#     'binary_mix': binary_lowess,
#     'linear_mix': linear_lowess,
#     'geometric_mix': geometric_lowess,
#     'amplitude_mix': amplitude_lowess,
#     'proposed_mix': proposedmix_lowess,
#     'jittering': jittered_ts,
#     'flipping': flipped_ts,
#     'scaling': scaled_ts,
#     'magnitude_warping': mag_warped_ts,
#     'permutation': permuted_ts,
    'stl_augment': stl_ts
}

# Create a MultiIndex DataFrame
keys = augmented_datasets.keys()
multi_index_df = pd.concat(augmented_datasets.values(), keys=keys, axis=1)

In [19]:
# Check the plots of original vs 
time_series_original = aapl['Close']
plot_augmented_ts(time_series_original, cut_lowess.Close, title='Cut_lowess Augmentation')
plot_augmented_ts(time_series_original, binary_lowess.Close, title='Binary_lowess Augmentation')
plot_augmented_ts(time_series_original, linear_lowess.Close, title='Linear_lowess Augmentation')
plot_augmented_ts(time_series_original, geometric_lowess.Close, title='Geometric_lowess Augmentation')
plot_augmented_ts(time_series_original, amplitude_lowess.Close, title='Amplitude_lowess Augmentation')
plot_augmented_ts(time_series_original, proposedmix_lowess.Close, title='Proposedmix_lowess Augmentation')

plot_augmented_ts(time_series_original, jittered_ts.Close, title='Jittering Augmentation')
plot_augmented_ts(time_series_original, flipped_ts.Close, title='Flipped Augmentation')
plot_augmented_ts(time_series_original, scaled_ts.Close, title='Scaled Augmentation')
plot_augmented_ts(time_series_original, mag_warped_ts.Close, title='Magnitude Warped Augmentation')
plot_augmented_ts(time_series_original, permuted_ts.Close, title='Permuted Augmentation')
plot_augmented_ts(time_series_original, stl_ts.Close, title='STL Augmentation')

In [20]:
from scipy.stats import ks_2samp
from scipy.signal import welch
from scipy.fft import fft
from sklearn.metrics import mutual_info_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Rules and tests with the original

To determine how much information is gone. we will use the following
- Mutual Information
Factors to Consider
Data Granularity: The frequency of your data (e.g., daily, weekly, monthly) affects how finely you might want to discretize it. More frequent data might benefit from more bins, but too many bins can lead to overfitting or spurious relationships.
    1. Volatility: Financial data can be highly volatile. More bins can capture nuances in volatile periods, but they should not capture noise as signal.
    2. Distribution Shape: The distribution of returns or price changes in financial data often deviates from normal, with heavy tails. A strategy that adapts to the data distribution can be more informative.
- KS Test
- Spectral Analysis
- MACD

In [21]:
def calculate_ema(prices, span):
    return prices.ewm(span=span, adjust=False).mean()

def calculate_macd(close_prices):
    ema_12 = calculate_ema(close_prices, 12)
    ema_26 = calculate_ema(close_prices, 26)
    macd = ema_12 - ema_26
    signal_line = calculate_ema(macd, 9)
    return macd, signal_line

def calculate_rsi(close_prices, periods=14):
    delta = close_prices.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

from sklearn.preprocessing import KBinsDiscretizer
def discretize_series(series, n_bins=20):
    # Assuming 'series' is a pandas Series
    series_reshaped = series.values.reshape(-1, 1)
    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    discretized = discretizer.fit_transform(series_reshaped).flatten()
    return discretized

In [22]:
def run_tests(original, augmented):
    # Distributional Analysis
    ks_stat, ks_pvalue = ks_2samp(original.Close, augmented.Close)

    # Spectral Analysis
    freqs, power_original = welch(original.Close)
    _, power_jittered = welch(augmented.Close)
    # Compare power_original and power_jittered

    # Mutual Information
    original_discretized = discretize_series(original.Close)
    augmented_discretized = discretize_series(augmented.Close)
    mi_score = mutual_info_score(original_discretized, augmented_discretized)
    
    # MACD and RSI visual inspection
    macd, signal_line = calculate_macd(original.Close)
    rsi = calculate_rsi(original.Close)
    augmented_macd, augmented_signal_line = calculate_macd(augmented.Close)
    augmented_rsi = calculate_rsi(augmented.Close)

    # Output results
    print(f"KS Statistic: {ks_stat}, P-value: {ks_pvalue}")
    print(f"Mutual Information Score: {mi_score}")
    
    results = {
        'KS Statistic': ks_stat,
        'P-value': ks_pvalue,
        'Mutual Information Score': mi_score,
        # Include any other metrics you calculate
    }
    
    plt.figure(figsize=(14, 10))
    plt.subplot(2, 1, 1)
    plt.plot(macd, label='Original MACD')
    plt.plot(signal_line, label='Original Signal Line')
    plt.plot(augmented_macd, label='Augmented MACD', linestyle='--')
    plt.plot(augmented_signal_line, label='Augmented Signal Line', linestyle='--')
    plt.title("MACD Comparison")
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(rsi, label='Original RSI')
    plt.plot(augmented_rsi, label='Augmented RSI', linestyle='--')
    plt.title("RSI Comparison")
    plt.legend()

    plt.show()
    
    
    return results

In [23]:
results_list = []
for method, dataset in augmented_datasets.items():
    result = run_tests(aapl, dataset)
    result['Method'] = method  # Add the method name to the results
    results_list.append(result)

# Convert the list of results into a DataFrame
results_df = pd.DataFrame(results_list)

# Get the list of all column names and remove 'Method'
columns = [col for col in results_df.columns if col != 'Method']

# Place 'Method' at the beginning of the list
ordered_columns = ['Method'] + columns

# Reindex the DataFrame with the new column order
results_df = results_df[ordered_columns]

### Forecasting

In [24]:
aapl['Return'] = np.log(aapl['Close']).diff()

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [26]:
# Define constants
TIME_STEPS = 20
ALPHA = 0.1  # Confidence level for prediction intervals

# Define the LSTM model creation function
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Prepare the data by scaling features and target
def prepare_data(dataframe, feature_columns, target_column):
    feature_scaler = StandardScaler()
    target_scaler = StandardScaler()
    
    scaled_features = feature_scaler.fit_transform(dataframe[feature_columns])
    scaled_target = target_scaler.fit_transform(dataframe[[target_column]].values)  
    
    return scaled_features, scaled_target.flatten(), feature_scaler, target_scaler

# Train the LSTM model and return it along with scalers and the test set
def train_evaluate_lstm(features, target, time_steps, epochs, batch_size):
    X, y = create_sequences(features, target, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    test_predictions = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    print(f"Test RMSE: {test_rmse}")
    
    return model, y_test, test_predictions, test_rmse

def create_sequences(features, target, time_steps):
    Xs, ys = [], []
    for i in range(len(features) - time_steps):
        Xs.append(features[i:(i + time_steps)])
        ys.append(target[i + time_steps])
    return np.array(Xs), np.array(ys)


def create_lstm_model(input_shape, lstm_units, dropout_rate, optimizer):
    model = Sequential()
    model.add(LSTM(lstm_units, return_sequences=True, input_shape=input_shape, dropout=dropout_rate))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Objective function for Optuna
def objective(trial, features, target, time_steps, batch_size):
    # Hyperparameters to be tuned by Optuna
    lstm_units = trial.suggest_categorical('lstm_units', [20, 50, 100])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Split the data
    X, y = create_sequences(features, target, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Model creation
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]), lstm_units, dropout_rate, Adam(learning_rate=learning_rate))
    
    # Model training
    model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=0)
    
    # Model evaluation
    test_predictions = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    
    return test_rmse

# Hyperparameter tuning with Optuna
def hyperparameter_tuning(features, target, time_steps, epochs, batch_size):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, features, target, time_steps, batch_size), n_trials=20)
    
    # Best hyperparameters
    best_params = study.best_params
    best_rmse = study.best_value
    print(f"Best RMSE: {best_rmse}")
    print(f"Best hyperparameters: {best_params}")

    # Retrain model with best hyperparameters
    best_model = create_lstm_model(
        (time_steps, features.shape[1]),
        best_params['lstm_units'],
        best_params['dropout_rate'],
        Adam(learning_rate=best_params['learning_rate'])
    )
    X, y = create_sequences(features, target, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    best_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    # Final evaluation
    test_predictions = best_model.predict(X_test)
    final_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    print(f"Final Test RMSE: {final_rmse}")

    return best_model, best_params, y_test, test_predictions, final_rmse

In [33]:
# Main code
augmented_results_lstm = {}

aapl.dropna(subset=['Return'], inplace=True)
for method, dataset in augmented_datasets.items():
    features = [col for col in dataset.columns if col not in ('Close', 'Adj Close', 'Return')]
    scaled_features, scaled_target, feature_scaler, target_scaler = prepare_data(aapl, features, 'Return')
    best_model, best_params, y_test, test_predictions, final_rmse = hyperparameter_tuning(scaled_features, scaled_target, TIME_STEPS, 100, 128)    
    
    augmented_results_lstm[method] = {
        'model': best_model,
        'params': best_params,
        'y_test': y_test,
        'predictions': test_predictions
    }

In [54]:
# Evaluation
for method, result in augmented_results_lstm.items():
    predictions = result['predictions']
    y_test = result['y_test']
    
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"Test RMSE for {method}: {rmse}")
    
    # Calculate errors
    errors = np.abs(pd.Series(predictions.flatten()) - pd.Series(y_test.flatten()))

    # Create a 1x2 subplot structure
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))

    # Plot Actual vs. Predicted Returns with Prediction Interval
    axs[0].plot(y_test, label='Actual Returns', color='blue')
    axs[0].plot(predictions, label='Predicted Returns', color='red')
    axs[0].set_title(f'Actual vs Predicted Returns with ({method})')
    axs[0].legend()

    # Plot the distribution of errors
    axs[1].hist(errors, bins=30, edgecolor='skyblue', density=True)
    axs[1].set_title(f'Error Distribution ({method})')
    axs[1].set_xlabel('Magnitude of Error')
    axs[1].set_ylabel('Frequency')

    plt.tight_layout()  # Adjust the layout
    plt.show()

In [69]:
augmented_results_lstm['cut_mix']['params']

In [84]:
df = augmented_datasets['cut_mix']
features = [col for col in dataset.columns if col not in ('Close', 'Adj Close', 'Return')]
scaled_features, scaled_target, feature_scaler, target_scaler = prepare_data(aapl, features, 'Return')
X, y = create_sequences(scaled_features, scaled_target, 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 # Model creation
model = create_lstm_model((X_train.shape[1], X_train.shape[2]), 50, 0.20462373842624892, Adam(learning_rate=0.07439518885658228))

# Model training
model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=0)

# Model evaluation
test_predictions = model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))


In [85]:
plt.plot(test_predictions[:50])
plt.plot(y_test[:50])
plt.legend()


In [78]:
# Create a 1x2 subplot structure
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

# Plot Actual vs. Predicted Returns with Prediction Interval
axs[0].plot(y_test, label='Actual Returns', color='blue')
axs[0].plot(test_predictions, label='Predicted Returns', color='red')
axs[0].set_title(f'Actual vs Predicted Returns with ({method})')
axs[0].legend()


In [39]:
augmented_results_lstm['cut_mix']['y_test']

In [40]:
X_train

# TCN

In [ ]:
from tcn import TCN

In [ ]:
def create_tcn_model(input_shape):
    input_layer = Input(shape=input_shape)
    tcn_layer = TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4], padding='causal', use_skip_connections=True)(input_layer)
    output_layer = Dense(1)(tcn_layer)
    model = Model(inputs=[input_layer], outputs=[output_layer])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Train the LSTM model and return it along with scalers and the test set
def train_evaluate_TCN(features, target, time_steps, epochs, batch_size):
    X, y = create_sequences(features, target, time_steps)
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = create_tcn_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    test_predictions = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    print(f"Test RMSE: {test_rmse}")
    
    return model, X_test, y_test

In [ ]:
# Train and evaluate LSTM model
tcn_model, X_test, y_test = train_evaluate_TCN(scaled_features, scaled_target, TIME_STEPS, 100, 128)

# Dictionary to store results for augmented datasets
augmented_results_tcn = {}
for method, dataset in augmented_datasets.items():
    # Ensure dataset is prepared with the correct columns
    new_data_prepared = dataset[features]  # Make sure dataset has the right columns
    
    predictions = predict_new_data(tcn_model, new_data_prepared, feature_scaler, target_scaler, TIME_STEPS)
        
    augmented_results_tcn[method] = {
        'Predictions': predictions
    }

In [ ]:
# Evaluation
for method, result in augmented_results_tcn.items():
    predictions = result['Predictions']
    actuals = scaled_target[TIME_STEPS-1:]  # Adjust as needed
    
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    print(f"Test RMSE for {method}: {rmse}")
    
    # Calculate errors
    errors = np.abs(pd.Series(predictions.flatten()) - pd.Series(actuals.flatten()))

    # Create a 1x2 subplot structure
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))

    # Plot Actual vs. Predicted Returns with Prediction Interval
    axs[0].plot(actuals, label='Actual Returns', color='blue')
    axs[0].plot(predictions, label='Predicted Returns', color='red')
    axs[0].set_title(f'Actual vs Predicted Returns ({method})')
    axs[0].legend()

    # Plot the distribution of errors
    axs[1].hist(errors, bins=30, edgecolor='skyblue', density=True)
    axs[1].set_title(f'Error Distribution ({method})')
    axs[1].set_xlabel('Magnitude of Error')
    axs[1].set_ylabel('Frequency')

    plt.tight_layout()  # Adjust the layout
    plt.show()